# Sesión 2: Exploración Avanzada y Modelado Básico con el Dataset de Clima

En esta sesión utilizaremos el dataset **cleaned_weather.csv** para:

- Realizar un análisis exploratorio avanzado (estadísticas descriptivas, visualizaciones y análisis de correlación).
- Preparar los datos para modelado (ajuste de tipos, selección de variables, división en entrenamiento y prueba).
- Entrenar un modelo de regresión lineal que prediga la **temperatura** a partir de otras variables (humedad, velocidad del viento, precipitación).

A lo largo de la notebook encontrarás explicaciones detalladas y ejercicios para que puedas ir practicando cada paso.


---

## 1. Importación de Librerías y Carga del Dataset

En este primer bloque cargaremos las librerías necesarias y leeremos el dataset **cleaned_weather.csv**.


---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de visualización para gráficos en línea
%matplotlib inline

# Cargar el dataset "dirty_cafe_sales.csv"
df = pd.read_csv("/cleaned_weather.csv")

# Mostrar dimensiones y primeras filas para conocer la estructura
print("Dimensiones del DataFrame:", df.shape)
display(df.head())


## 2. Exploración Inicial del Dataset

Observa las columnas y tipos de datos. En este ejemplo, asumiremos que las columnas son:
- **date**: Fecha (string)
- **temperature**: Temperatura (numérica) → Variable objetivo
- **humidity**: Humedad (numérica)
- **wind_speed**: Velocidad del viento (numérica)
- **precipitation**: Precipitación (numérica)

Ejercicio: Revisa la salida de `df.info()` y comenta en el chat si todos los tipos son los esperados.


---

In [ ]:
# Revisar la información general del DataFrame
df.info()

# Mostrar estadísticas descriptivas para las columnas numéricas
display(df.describe))




El dataset contiene 21 columnas, entre las que destacan:
- **date**: Fecha en formato string.
- **T**: Temperatura (variable que queremos predecir).
- **p**: Presión.
- **Tpot**: Temperatura potencial.
- **Tdew**: Temperatura del punto de rocío.
- **rh**: Humedad relativa.
- **VPmax, VPact, VPdef**: Variables relacionadas con la presión de vapor.
- **sh**: Humedad específica.
- **H2OC**: Concentración de vapor de agua.
- **rho**: Densidad.
- **wv** y **max. wv**: Velocidad del viento.
- **wd**: Dirección del viento.
- **rain**: Precipitación.
- **raining**: Indicador de lluvia.
- **SWDR**: Radiación solar de onda corta.
- **PAR** y **max. PAR**: Radiación fotosintéticamente activa.
- **Tlog**: Temperatura logarítmica (u otra transformación de T).

---

## 3. Ajuste de Tipos y Conversión de la Columna de Fecha

Es importante trabajar con fechas en formato datetime para aprovechar herramientas de series de tiempo. Convirtamos la columna **date** a formato datetime.

Ejercicio: ¿Qué formato de fecha crees que tiene el archivo? (Por ejemplo, "YYYY-MM-DD"). Si no es ese, ajusta el parámetro `format`.


---

In [ ]:
type(df['date'])
df['date'].dtype
print(type(df['date'].iloc[0]))
print(df[date].dtype)
prueba = df.copy()  ###### NO QUIERO CAMBIAR el dataframe que tengo guardado como 'df'

In [ ]:
# Convertir la columna "date" a formato datetime
#df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d', errors='coerce')
prueba['date'] = pd.to_datetime(prueba['date'])
print("Tipo de 'date' después de la conversión:", prueba['date'].dtype)

# Opcional: Ordenar el DataFrame por fecha
prueba = prueba.sort_values('date')
display(prueba.head())


## 4. Análisis Exploratorio Avanzado (EDA)

Realizaremos:
- Análisis descriptivo y visualizaciones para entender la distribución de la **temperatura** (`T`).
- Exploración de relaciones entre `T` y otras variables (por ejemplo, `rh`, `rain`, `wv`, `SWDR` y `PAR`).
- Análisis de correlación mediante un heatmap.

Ejercicio: Observa el histograma de la temperatura y un scatter plot entre la humedad relativa (`rh`) y la temperatura (`T`). ¿Notas alguna tendencia?


---

In [ ]:
# Histograma de la Temperatura
plt.figure(figsize=(8, 5))
plt.hist(df['Temperatura'].dropna(), bins=20, color='skyblue', edgecolor='black')
plt.title('Distribución de la Temperatura (T)')
plt.xlabel('Temperatura (T)')
plt.ylabel('Frecuencia')
plt.show()

# Scatter plot: Humedad relativa (rh) vs Temperatura (T)
plt.figure(figsize=(8, 5))
plt.scatter(df['rh'], df['T'], alpha=0.6, color='orange')
plt.title('Humedad Relativa (rh) vs Temperatura (T)')
plt.xlabel('Humedad Relativa (rh)')
plt.ylabel('Temperatura (T)')
plt.show()



### Análisis de Correlación

Calculemos la matriz de correlación entre las variables numéricas para identificar relaciones potenciales, y visualicémosla con un heatmap.

---

In [ ]:
# Seleccionar las columnas numéricas de interés
cols_numericas = ['T', 'presión', 'Tpot', 'Tdew', 'rh', 'VPmax', 'VPact', 'VPdef', 'sh', 'H2OC', 'rho', 'wv', 'max. wv', 'wd', 'rain', 'raining', 'SWDR', 'PAR', 'max. PAR', 'Tlog']
df_num = df[cols_numericas]

# Calcular la matriz de correlación
corr_matrix = df_num.corr()

# Visualizar la matriz de correlación con un heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title('Matriz de Correlación entre Variables')
plt.show()


## Análisis de Series de Tiempo

Ahora que hemos analizado las correlaciones entre variables, aprovecharemos la columna **date** para explorar la evolución de la temperatura (**T**) a lo largo del tiempo.

**Pasos a seguir:**
1. **Establecer la columna `date` como índice:** Esto nos permite utilizar las funcionalidades de resample y slicing basadas en fechas.
2. **Re-muestrear los datos:** Agruparemos la temperatura por períodos (por ejemplo, mensual) para observar tendencias y patrones estacionales.
3. **Visualizar la serie de tiempo:** Graficaremos la temperatura media por mes para identificar tendencias generales.

**Ejercicio:**  
- Modifica el período de resample (por ejemplo, semanal o anual) y observa cómo cambia la visualización.  
- Intenta calcular una media móvil para suavizar la serie y destacar las tendencias a largo plazo.

---

In [57]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
# Creamos una copia del DataFrame para el análisis de series de tiempo
df_ts = df.copy()

# Establecer la columna "date" como índice (si no lo está)
df_ts.set_index('date', inplace=True)

# Resample: calcular la temperatura media mensual
temp_mensual = df_ts['T'].resample('M').mean()

# Graficar la serie de tiempo de la temperatura media mensual
plt.figure(figsize=(12, 6))
plt.plot(temp_mensual.index, temp_mensual, marker='o', linestyle='-')
plt.title('Temperatura Media Mensual')
plt.xlabel('Fecha')
plt.ylabel('Temperatura (T)')
plt.grid(True)
plt.show()

# Ejercicio adicional: calcular y graficar una media móvil (por ejemplo, de 3 meses)
temp_mensual_mm = temp_mensual.rolling(window=3).mean()
plt.figure(figsize=(12, 6))
plt.plot(temp_mensual.index, temp_mensual, label='Temperatura Mensual', alpha=0.5)
plt.plot(temp_mensual_mm.index, temp_mensual_mm, label='Media Móvil (3 meses)', color='red')
plt.title('Temperatura Mensual y Media Móvil de 3 Meses')
plt.xlabel('Fecha')
plt.ylabel('Temperatura (T)')
plt.legend()
plt.grid(True)
plt.show()


## 5. Preparación de Datos para el Modelado

El objetivo es predecir la **temperatura** (`T`). Seleccionaremos algunas variables predictoras relevantes. Para este ejemplo usaremos:
- **rh** (humedad relativa)
- **rain** (precipitación)
- **wv** (velocidad del viento)
- **SWDR** (radiación solar de onda corta)
- **PAR** (radiación fotosintéticamente activa)

Ejercicio: Verifica si existen valores nulos en estas columnas y, de ser necesario, rellena o elimina esas filas.

---

## ¿En qué consiste el modelado y la separación de datos?

**Modelado:**  
El modelado en machine learning es el proceso de entrenar un algoritmo para que aprenda patrones a partir de los datos. La idea es que, una vez "aprendido", el modelo pueda predecir o clasificar datos nuevos que no ha visto durante el entrenamiento. Por ejemplo, en un problema de regresión, el modelo intenta predecir un valor numérico (como la temperatura) basándose en variables de entrada.

**Separación de Datos de Entrenamiento y Prueba:**  
Para evaluar la capacidad de generalización del modelo (es decir, qué tan bien funciona en datos no vistos), dividimos el conjunto de datos en dos partes:

- **Conjunto de Entrenamiento:**  
  Se utiliza para **ajustar el modelo**, es decir, para que el algoritmo encuentre los patrones y ajuste sus parámetros internamente.  
- **Conjunto de Prueba:**  
  Se reserva para **evaluar el rendimiento** del modelo una vez entrenado. Esto nos permite medir cómo se comporta el modelo con datos nuevos y estimar su capacidad de generalización.

**Qué hace scikit-learn en cada parte:**

- **train_test_split:**  
  Scikit-learn proporciona la función `train_test_split` para dividir los datos de forma aleatoria en los conjuntos de entrenamiento y prueba, siguiendo una proporción definida (por ejemplo, 80% entrenamiento y 20% prueba).

- **Entrenamiento (fit):**  
  Cuando llamamos a `modelo.fit(X_train, y_train)`, scikit-learn utiliza el conjunto de entrenamiento para ajustar los parámetros del modelo. El algoritmo "aprende" de estos datos y establece internamente la relación entre las características (X) y la variable objetivo (y).

- **Predicción (predict):**  
  Luego, al usar `modelo.predict(X_test)`, el modelo utiliza los parámetros aprendidos para hacer predicciones sobre el conjunto de prueba.

- **Evaluación:**  
  Finalmente, se comparan las predicciones (`y_pred`) con los valores reales (`y_test`) usando métricas de evaluación (como RMSE o R² en problemas de regresión). Esto nos indica qué tan bien el modelo puede generalizar a datos nuevos.

En resumen, el proceso de modelado con scikit-learn sigue estos pasos:
1. Dividir los datos en entrenamiento y prueba.
2. Entrenar el modelo usando el conjunto de entrenamiento.
3. Predecir y evaluar el modelo usando el conjunto de prueba.

Esta metodología nos ayuda a evitar el sobreajuste y a tener una estimación realista del desempeño del modelo en situaciones del mundo real.

---

## ¿Qué forma tiene el modelo de regresión lineal que creamos?

El modelo que construimos es un **modelo de regresión lineal**, el cual se representa mediante una ecuación lineal. Su forma general es:

$$
\hat{y} = \beta_0 + \beta_1 \cdot x_1 + \beta_2 \cdot x_2 + \dots + \beta_n \cdot x_n
$$

donde:

- $\hat{y}$ es el valor predicho de la variable objetivo (en nuestro caso, la temperatura "T").
- $\beta_0$ es el **término de intersección** o **bias**; representa el valor de $\hat{y}$ cuando todas las características $x_i$ son cero.
- $\beta_1, \beta_2, \dots, \beta_n$ son los **coeficientes** o **pesos** asociados a cada característica $(x_1, x_2, \dots, x_n)$. Cada uno indica cuánto influye la correspondiente variable predictora en el valor de $(\hat{y})$.

En nuestro ejemplo, si usamos las variables predictoras `rh`, `rain`, `wv`, `SWDR` y `PAR`, el modelo toma la forma:

$$
\text{T} = \beta_0 + \beta_1 \cdot \text{rh} + \beta_2 \cdot \text{rain} + \beta_3 \cdot \text{wv} + \beta_4 \cdot \text{SWDR} + \beta_5 \cdot \text{PAR}
$$

El proceso de entrenamiento (usando `modelo.fit(X_train, y_train)`) consiste en encontrar los valores óptimos de $(\beta_0, \beta_1, \dots, \beta_5)$ que minimicen el error entre las predicciones $(\hat{y})$ y los valores reales $(y)$. Una vez ajustado, el modelo puede predecir la temperatura para nuevos valores de las características.

---

In [ ]:
# Seleccionar las variables predictoras y la variable objetivo
features = ['rh', 'rain', 'wv', 'SWDR', 'PAR']
target = 'T'

# Crear un DataFrame para modelado y eliminar filas con valores nulos en las columnas de interés
df_model = df[features + [target]].dropna()

print("Dimensiones del DataFrame para modelado:", df_model.shape)
display(df_model.head())


## 6. División en Conjuntos de Entrenamiento y Prueba

Dividiremos el dataset en:
- **80%** para entrenamiento.
- **20%** para prueba.

Ejercicio: Realiza la división y comprueba las dimensiones de cada subconjunto.

---

In [ ]:
from sklearn.model_selection import train_test_split

X = df_model[features]
y = df_model[target]

# Dividir en entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Dimensiones de X_train:", X_train.shape)
print("Dimensiones de X_test:", X_test.shape)


## 7. Construcción y Entrenamiento del Modelo de Regresión Lineal

Utilizaremos un modelo de regresión lineal para predecir la temperatura (`T`) a partir de las variables seleccionadas.

Pasos:
- Crear el modelo.
- Entrenarlo con el conjunto de entrenamiento.
- Predecir sobre el conjunto de prueba.
- Evaluar el desempeño usando RMSE y el coeficiente de determinación (R²).

Ejercicio: Entrena el modelo y observa los resultados.


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Crear y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train.values.ravel())  # Aseguramos que y_train sea 1D

# Predecir en el conjunto de prueba
y_pred = modelo.predict(X_test)

# Calcular el MSE sin el parámetro 'squared'
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)  # Calculamos la raíz cuadrada para obtener el RMSE

# Calcular el coeficiente de determinación R²
r2 = r2_score(y_test, y_pred)

print("RMSE:", rmse)
print("R²:", r2)


## 8. Conclusiones y Ejercicios Finales

En esta sesión hemos:
- Explorado en profundidad el dataset meteorológico, analizando su estructura, distribuciones y relaciones.
- Convertido la columna **date** a formato datetime para un manejo correcto de la información temporal.
- Preparado los datos seleccionando las variables predictoras y eliminando filas con valores nulos.
- Dividido el dataset en conjuntos de entrenamiento y prueba.
- Construido y evaluado un modelo de regresión lineal para predecir la temperatura.

**Ejercicios para profundizar:**
1. **Visualización Adicional:**  
   - Realiza un scatter plot entre **wv** (velocidad del viento) y **T** y comenta la relación.
   - Grafica la evolución de la temperatura a lo largo del tiempo usando la columna **date**.
2. **Prueba con Otros Modelos:**  
   - Sustituye la regresión lineal por un modelo de árbol de decisión (DecisionTreeRegressor) y compara las métricas.
3. **Ajuste de Hiperparámetros:**  
   - Experimenta variando las variables predictoras (por ejemplo, añade **Tdew** o **Tpot**) y observa cómo mejora o empeora el desempeño.

Estos ejercicios te ayudarán a profundizar en el análisis de datos meteorológicos y en la construcción de modelos de machine learning básicos.
